In [2]:
import requests
import os
from io import BytesIO
from zipfile import ZipFile

In [3]:
# Official page: https://gtfs.adelaidemetro.com.au/
# URLs
base_url = "http://gtfs.adelaidemetro.com.au/v1"
latest_version_number = "static/latest/version.txt"
latest_version_feed = "static/latest/google_transit.zip"

# binary data as response
request_headers={"Content-Type": "application/octet-stream"}

# Feed template
# practice: how to use string template + format
feed_url_template = "static/{version}/google_transit.zip"

# output
dest_path = 'destination'
os.makedirs(dest_path, exist_ok=True)

Read latest version

In [ ]:
# 1. Find the latest feed version
# r = requests.get(f'{base_url}/{latest_version_number}')
# r.__dict__

# 2. Download latest GTFS feed version using request
r = requests.get(f'{base_url}/{latest_version_feed}',headers=request_headers)
r.__dict__
with open(f'{dest_path}/latest.zip','wb') as f:
    for chunk in r.iter_content(chunk_size=128):
        f.write(chunk)
# # Feed's URL

{'_content': b'1603',
 '_content_consumed': True,
 '_next': None,
 'status_code': 200,
 'headers': {'Content-Type': 'binary/octet-stream', 'Content-Length': '4', 'Connection': 'keep-alive', 'Date': 'Thu, 04 Dec 2025 11:14:13 GMT', 'Last-Modified': 'Thu, 04 Dec 2025 03:48:19 GMT', 'ETag': '"f3173935ed8ac4bf073c1bcd63171f8a"', 'x-amz-server-side-encryption': 'AES256', 'x-amz-version-id': 'dZEdZHWjOq0mKZtg5yyqO4zX7FPv3K4w', 'Accept-Ranges': 'bytes', 'Server': 'AmazonS3', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 ce64651cc74efc9068c2256f6672ca24.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'SYD3-P3', 'X-Amz-Cf-Id': 'kby-ldH5-Cin9mxSavxMqQZhmY0CQ_i15sxDIu5iUaSHAIfSr5CIVA==', 'Age': '226'},
 'raw': <urllib3.response.HTTPResponse at 0x1907c167a30>,
 'url': 'https://gtfs.adelaidemetro.com.au/v1/static/latest/version.txt',
 'encoding': None,
 'history': [<Response [301]>],
 'reason': 'OK',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(microseconds=272817),
 'request': <

Dynamic read multiple feed versions: 930 to 939

In [11]:
# HINTS
# 1. For loop the versions
# 2. Generate versioned URL
# 3. Request and handle response

os.makedirs('downloaded_files',exist_ok=True)
for version in list(range(930,940)):
    # print(f'{base_url}/static/{version}/google_transit.zip')
    r = requests.get(f'{base_url}/static/{version}/google_transit.zip',headers=request_headers, stream=True)
    # print(r.ok)
    if(r.ok):
        with open(f'downloaded_files/{version}.zip','wb') as f:
            for chunk in r.iter_content(chunk_size=128):
                f.write(chunk)
            print(f'version {version} downloaded')    
    else:
        print('failed to make request')



version 930 downloaded
version 931 downloaded
version 932 downloaded
version 933 downloaded
version 934 downloaded
version 935 downloaded
version 936 downloaded
version 937 downloaded
version 938 downloaded
version 939 downloaded


Unzip feed versions & export files

In [13]:
# # using ZipFile to read zipfile
# # then extract all

# # Define the path to your zip file
# zip_file_path = 'destination/latest.zip' 
zip_file_path = 'downloaded_files'

# # Define the directory where you want to extract the files (optional)
extract_to_path = 'downloaded_files/unzip' 
# # If not specified, files will be extracted to the current working directory
# extract_to_path = 'destination/latest/unzip/' 
os.makedirs(extract_to_path,exist_ok=True)

for version in range(930,940):
    with ZipFile(f'{zip_file_path}/{version}.zip', 'r') as zip_obj:
        # Extract all files to the specified directory
        zip_obj.extractall(f'{extract_to_path}/{version}')



Extra: extract zipfile from response.content

In [ ]:
# HINTS
# Use BytesIO (file bytes in memory) + ZipFile
# dung bytesio store bytes from requests(content) on RAM va dung zipfile to unzip va luu file vao ma'y
bytesIO_path = 'bytesIO_files'
os.makedirs(bytesIO_path,exist_ok=True)

for version in range(930,940):
    r = requests.get(f'{base_url}/static/{version}/google_transit.zip',headers=request_headers, stream=True)

    if(r.ok):
        bytes_data = BytesIO(r.content)
        with ZipFile(bytes_data,'r') as obj:
            obj.extractall(f'{bytesIO_path}/{version}/')
        print(f'{version} extracted sucessfully')    
    else:
        print('failed to extract')            